<a href="https://colab.research.google.com/github/Janina712/MLTSA_FinalProject/blob/main/Preprocess_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install mne matplotlib

     |████████████████████████████████| 7.5 MB 5.0 MB/s 


In [2]:
import numpy as np
import mne 
import pandas as pd
import os
import pylab as pl

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
cd /content/drive/MyDrive/MLTSA22/Final_Project

/content/drive/MyDrive/MLTSA22/Final_Project


In [5]:
!ls /content/drive/MyDrive/MLTSA22/Final_Project

00000258_s002_t000.edf
all_tuh_eeg_seizure_links.txt
dev__01_tcp_ar__002__00000258__s002_2003_07_21__00000258_s002_t000.edf
dev__01_tcp_ar__002__00000258__s003_2003_07_22__00000258_s003_t001.edf


In [34]:
file = ['00000258_s002_t000.edf', 'dev__01_tcp_ar__002__00000258__s002_2003_07_21__00000258_s002_t000.edf',
        'dev__01_tcp_ar__002__00000258__s003_2003_07_22__00000258_s003_t001.edf']

In [32]:
def preprocessing(file):
  #read the data
  data = mne.io.read_raw_edf(file)
  data = data.copy().load_data()
  #notch filter 60 Hz - to get rid of the noise because of the power outlets
  eeg_picks = mne.pick_types(data.info, eeg=True)
  freqs = []
  for i in range(0, int(data.info['sfreq']/2), 60):
    if i != 0:
      freqs.append(i)
  data_notch = data.notch_filter(freqs=np.asarray(freqs), picks=eeg_picks)

  #high pass filter
  data_filter = data_notch.filter(l_freq=0.2, h_freq=None)

  #downsampling
  data_done = data_filter.resample(sfreq=200)
  return data_done


'''

  #raw_data = data.get_data()
  # you can get the metadata included in the file and a list of all channels:
  info = data.info
  channels = data.ch_names
  info['chs']
  
  freqs = (60, 120, 180)
  raw_notch = data.copy().load_data()
  raw_notch = raw_notch.notch_filter(freqs=freqs, picks=eeg_picks)
  #high-pass filter
raw_notch = raw_notch.filter(l_freq=0.2, h_freq=None)
fig = raw_notch.plot_psd(fmax=20, average=True)
#downsampling
raw_downsampled = raw_notch.resample(sfreq=200)
fig = raw_downsampled.plot_psd(average=True)'''

"\n\n  #raw_data = data.get_data()\n  # you can get the metadata included in the file and a list of all channels:\n  info = data.info\n  channels = data.ch_names\n  raw_data.shape\n  info['sfreq'] #the way to extract sampling frequency\n  info['chs']\n  \n  freqs = (60, 120, 180)\n  raw_notch = data.copy().load_data()\n  raw_notch = raw_notch.notch_filter(freqs=freqs, picks=eeg_picks)\n  #high-pass filter\nraw_notch = raw_notch.filter(l_freq=0.2, h_freq=None)\nfig = raw_notch.plot_psd(fmax=20, average=True)\n#downsampling\nraw_downsampled = raw_notch.resample(sfreq=200)\nfig = raw_downsampled.plot_psd(average=True)"

In [46]:
data = preprocessing(file[0])
df = pd.DataFrame(data.get_data())
df['channel'] = data.ch_names


Extracting EDF parameters from /content/drive/MyDrive/MLTSA22/Final_Project/00000258_s002_t000.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 7999  =      0.000 ...    19.997 secs...
Setting up band-stop filter

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower transition bandwidth: 0.50 Hz
- Upper transition bandwidth: 0.50 Hz
- Filter length: 2641 samples (6.603 sec)

Filtering raw data in 1 contiguous segment
Setting up high-pass filter at 0.2 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal highpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.20
- Lower transition bandwidth: 0.20 Hz (-6 dB c

In [47]:
df

,0,1,2,3,4,5,6,7,8,9,...,3991,3992,3993,3994,3995,3996,3997,3998,3999,channel
0,-4.379170e-09,-1.009273e-05,-1.347385e-05,-1.034604e-05,-1.390651e-05,-2.185444e-05,-1.449920e-05,-1.295677e-05,-2.266596e-05,-2.249664e-05,...,-8.405200e-06,-3.178715e-06,2.723583e-06,-3.098573e-06,-6.916099e-06,6.546581e-07,-6.808843e-07,-0.000007,-4.472419e-07,EEG FP1-REF
1,-4.454867e-09,-1.203555e-05,-1.571610e-05,-1.169151e-05,-1.874109e-05,-2.526137e-05,-1.973138e-05,-1.970534e-05,-2.528652e-05,-2.826188e-05,...,1.813434e-06,3.547278e-06,6.277272e-06,2.339261e-06,-2.000302e-06,5.867000e-06,1.301726e-07,-0.000005,-7.109538e-07,EEG FP2-REF
2,-2.859807e-08,-5.337285e-06,-3.590916e-06,-8.300046e-06,-5.814302e-06,-1.352105e-05,-6.341331e-06,-7.229655e-06,-1.409014e-05,-1.441935e-05,...,-9.198560e-06,-1.028235e-05,-4.503042e-06,-2.036565e-06,-6.783345e-06,1.236061e-06,-2.772401e-06,-0.000006,-1.776618e-06,EEG F3-REF
3,-8.383492e-09,-4.327637e-06,-3.679265e-06,3.522454e-06,3.090580e-06,9.815249e-07,7.122793e-06,1.068468e-05,4.987821e-06,4.461822e-06,...,7.137624e-06,9.258681e-06,1.096617e-05,6.545505e-06,3.096971e-06,5.500807e-06,1.879165e-06,-0.000002,1.937063e-07,EEG F4-REF
4,-6.380791e-11,-5.142869e-06,-8.740026e-06,-3.024007e-06,-1.865686e-06,-8.697765e-06,-5.271309e-06,-4.620601e-06,-8.167300e-06,-6.812171e-06,...,-1.425835e-05,-1.004348e-05,-5.645517e-06,-8.906732e-06,-9.178801e-06,-4.094146e-06,-4.978648e-06,-0.000006,-2.283970e-06,EEG C3-REF
5,-6.055977e-09,-3.818833e-06,-4.950422e-06,4.019427e-06,5.062312e-06,4.531843e-06,1.319236e-05,1.819210e-05,1.646563e-05,2.129534e-05,...,-1.649715e-06,3.346730e-06,4.641957e-06,2.582470e-06,8.718696e-07,2.829982e-06,-1.922543e-06,-0.000004,-2.184437e-07,EEG C4-REF
6,-2.144383e-09,-2.975665e-06,6.198995e-08,6.254080e-06,7.116523e-06,-8.497132e-07,6.832730e-07,2.494384e-07,-5.968640e-06,-3.075387e-06,...,-1.305263e-05,-9.446572e-06,-5.382057e-06,-5.605893e-06,-5.111746e-06,1.114806e-06,-2.802315e-06,-0.000002,1.080375e-07,EEG P3-REF
7,5.818894e-09,-4.777208e-06,-3.787798e-06,4.753185e-06,4.517842e-06,2.297280e-06,1.110051e-05,1.506185e-05,1.396597e-05,1.931037e-05,...,-1.328754e-06,2.233816e-06,3.202839e-06,1.576178e-06,1.955434e-07,3.505971e-06,-2.093292e-06,-0.000006,-6.609122e-07,EEG P4-REF
8,-4.091781e-09,-8.453208e-07,2.957728e-06,1.401100e-05,1.445923e-05,9.616097e-06,8.250557e-06,4.780501e-06,-4.837165e-06,-3.092542e-06,...,-1.406388e-05,-1.404780e-05,-9.000811e-06,-9.190747e-06,-7.173895e-06,-3.548639e-07,-4.704921e-06,-0.000003,8.348308e-07,EEG O1-REF
9,-8.666753e-09,-5.447246e-06,-4.237754e-06,5.730911e-06,4.320453e-06,2.684849e-07,2.860162e-06,2.611079e-06,-4.511130e-06,-1.368642e-06,...,3.127496e-06,1.611177e-06,2.442229e-06,8.184202e-07,-2.061453e-06,3.027926e-06,-2.480998e-06,-0.000007,-5.712261e-07,EEG O2-REF


In [49]:
df.columns

Index([        0,         1,         2,         3,         4,         5,
               6,         7,         8,         9,
       ...
            3991,      3992,      3993,      3994,      3995,      3996,
            3997,      3998,      3999, 'channel'],
      dtype='object', length=4001)